In [1]:
import requests
from urllib.parse import urlencode
from bs4 import BeautifulSoup
import json
import pyperclip
import psycopg2
from requests.exceptions import ChunkedEncodingError, RequestException
from collections import defaultdict

In [2]:
headers = {
    'authority': 'moto.av.by',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'accept-language': 'ru,en;q=0.9',
    'cache-control': 'no-cache',
    'pragma': 'no-cache',
    'sec-ch-ua': '"Not_A Brand";v="8", "Chromium";v="120", "YaBrowser";v="24.1", "Yowser";v="2.5"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 YaBrowser/24.1.0.0 Safari/537.36',
}
    # если теперь что-то будет не так, то попробовать requests.Session()
    #подставить ключи если будет надо
    #"X-Api-Key": "-",
    #"X-User-Group": "-",

In [12]:
# искомый урл
url = 'https://moto.av.by/bike/victory/vegas/118836109'

In [13]:
# Получение json
try:
    response = requests.get(url, headers=headers)
    # Дальнейшая обработка успешного запроса
except SSLError:
    # Обработка ошибки SSL
    print(f"Произошла ошибка SSL при обращении к URL: {url}. Пропускаем выполнение для данного URL.")
except requests.exceptions.ConnectionError as e:
    print("Ошибка подключения:", e)

except Exception as e:
    print(f"Произошла ошибка при обработке id {id_value}: {e}")
    broken_link_count += 1

src = response.text
soup = BeautifulSoup(src, 'lxml')

src = response.text 
soup = BeautifulSoup(src, 'lxml')
script_element = soup.find("script", id="__NEXT_DATA__") #Достаем жсон
json_string = script_element.string #Конвертируем жсон в стринг
data = json.loads(json_string) #Пакуем в data

In [5]:
# скопировать тест json'а в буфер обмена
pyperclip.copy(data)

In [14]:
# Подключение к базе
with open('config.json') as file:
    config = json.load(file)

mail_login = config['sender login']
mail_password = config['sender password']
pgre_login = config['postgre login']
pgre_password = config['postgre password']
pgre_host = config['postgre host']
pgre_port = config['postgre port']
pgre_db = config['postgre database']
recipients = config['mail recipients']

#Подключение к postgres
conn = psycopg2.connect(
    host = pgre_host,
    port = pgre_port,
    database = pgre_db,
    user = pgre_login,
    password = pgre_password
)

In [15]:
# получить переменные
properties = data['props']['initialState']['advert']['advert']['properties']
brand = next((prop['value'] for prop in properties if prop['name'] == 'brand'), None)
model = next((prop['value'] for prop in properties if prop['name'] == 'model'), None) 
year  = next((prop['value'] for prop in properties if prop['name'] == 'year'), None)
mtype  = next((prop['value'] for prop in properties if prop['name'] == 'purpose_type'), None)
cylcount = next((prop['value'] for prop in properties if prop['name'] == 'cylinder_number'), None)
capacity = next((prop['value'] for prop in properties if prop['name'] == 'engine_capacity'), None)
seller = data['props']['initialState']['advert']['advert']['sellerName']
location = data['props']['initialState']['advert']['advert']['locationName']
mileage = next((prop['value'] for prop in properties if prop['name'] == 'mileage_km'), None)
id = data['props']['initialState']['advert']['advert']['id']


In [16]:
# Функция "ручной" проверки дубликатов
def duplicates_manual_check(id, brand, model, year, mtype, cylcount, capacity, seller, location, mileage):
    duplcursor = conn.cursor()
    query = """
        SELECT id, url, duplicate_flag, duplicate_id
        FROM public.av_full
        WHERE brand = %(brand)s
          AND model = %(model)s
          AND year = %(year)s
          AND type = %(type)s
          AND cylinders = %(cylinders)s
          AND capacity BETWEEN %(min_capacity)s AND %(max_capacity)s
          AND LOWER(seller) = %(seller)s
          AND locations = %(location)s
          AND mileage BETWEEN %(min_mileage)s AND %(max_mileage)s
        AND id != %(id)s
        ORDER BY date ASC;
    """
    params = {
        'id': id,
        'min_mileage': int(mileage) * 0.7,
        'max_mileage': int(mileage) * 1.3,
        'min_capacity': int(capacity) * 0.9,
        'max_capacity': int(capacity) * 1.1,
        'brand': brand, 
        'model': model,
        'year': year,
        'type': mtype,
        'cylinders': cylcount,
        'seller':seller.lower(),
        'location': location
    }
    duplcursor.execute(query, params)
    dmc_results = duplcursor.fetchall()

    duplcursor.close()

    return dmc_results

In [17]:
dmc_results = duplicates_manual_check(id, brand, model, year, mtype, cylcount, capacity, seller, location, mileage)

In [18]:
dmc_is_duplicate = []
dmc_not_duplicate = []
for i in dmc_results:
    if i[2] is True:
        dmc_is_duplicate.append(i)
        
    else:
        dmc_not_duplicate.append(i)

In [ ]:
#АКТУАЛЬНАЯ ВЕРСИЯ

# Дополнение хтмл отчета строчкой про дубликаты если они есть
if len(dmc_not_duplicate or dmc_is_duplicate) > 0:
    duplicate_html_block = f"""<p style="text-align: center;"><strong>Найдены дубликаты (<span style="color: #ff6600;">вероятные</span> / <span style="color: #008000;">старые</span>):</strong></p>"""
    # Блок дубликатов которые duplicate = False
    for idx, x in enumerate(dmc_not_duplicate):
        m_d_id = x[0]
        m_d_url = x[1]
        duplicate_html_block += f"""<a href="{m_d_url}" style="color: #ff6600;">{m_d_id}</a>"""
        # Добавляем запятую, если это не последний элемент из обоих списков
        if idx != len(dmc_not_duplicate) - 1 or len(dmc_is_duplicate) > 0:
            duplicate_html_block += ", "

    # Блок дубликатов которые duplicate = True
    for idx, x in enumerate(dmc_is_duplicate):
        m_d_id = x[0]
        m_d_url = x[1]
        m_d_dupl_id = x[3]
        duplicate_html_block += f"""<a href="{m_d_url}" style="color: #008000;">{m_d_id}</a> ({m_d_dupl_id})"""
        # Добавляем запятую, если это не последний элемент
        if idx != len(dmc_is_duplicate) - 1:
            duplicate_html_block += ", "

In [ ]:
# Новая версия
# Дополнение хтмл отчета строчкой про дубликаты если они есть
if len(dmc_not_duplicate or dmc_is_duplicate) > 0:
    duplicate_html_block = f"""<p style="text-align: center;"><strong>Найдены дубликаты (<span style="color: #ff6600;">вероятные</span> / <span style="color: #008000;">старые</span>):</strong></p>"""
    # Блок дубликатов которые duplicate = False
    for idx, x in enumerate(dmc_not_duplicate):
        m_d_id = x[0]
        m_d_url = x[1]
        duplicate_html_block += f"""<a href="{m_d_url}" style="color: #ff6600;">{m_d_id}</a>"""
        # Добавляем запятую, если это не последний элемент из обоих списков
        if idx != len(dmc_not_duplicate) - 1 or len(dmc_is_duplicate) > 0:
            duplicate_html_block += ", "


# Блок дубликатов которые duplicate = True
duplicates_grouped = defaultdict(list)
for item in dmc_is_duplicate:
    m_d_id = item[0]        # например, 107069185
    m_d_url = item[1]       # ссылка
    m_d_dupl_id = item[3]   # например, 109140984

    # Добавляем ID как ссылку, но храним только ID для вывода
    link = f'<a href="{m_d_url.strip()}" style="color: #008000;">{m_d_id}</a>'
    duplicates_grouped[m_d_dupl_id].append(link)

if duplicates_grouped:
    for dupl_id, ids in duplicates_grouped.items():
        duplicate_html_block += f"<br>Для id {dupl_id}:<br>"
        duplicate_html_block += ", ".join(ids)
else:
    duplicate_html_block += "не найдено."

In [33]:
duplicate_html_block

'<p style="text-align: center;"><strong>Найдены дубликаты (<span style="color: #ff6600;">вероятные</span> / <span style="color: #008000;">старые</span>):</strong>:</p><a href="https://moto.av.by/bike/victory/vegas/102220235" style="color: #ff6600;">102220235</a>, <a href="https://moto.av.by/bike/victory/vegas/102618014" style="color: #ff6600;">102618014</a>, <a href="https://moto.av.by/bike/victory/vegas/103361628" style="color: #ff6600;">103361628</a>, <a href="https://moto.av.by/bike/victory/vegas/103729884" style="color: #ff6600;">103729884</a>, <a href="https://moto.av.by/bike/victory/vegas/104161878" style="color: #ff6600;">104161878</a>, <a href="https://moto.av.by/bike/victory/vegas/104374203" style="color: #ff6600;">104374203</a>, <a href="https://moto.av.by/bike/victory/vegas/106853897" style="color: #ff6600;">106853897</a>, <a href="https://moto.av.by/bike/victory/vegas/108355823" style="color: #ff6600;">108355823</a>, <a href="https://moto.av.by/bike/victory/vegas/116772778"

In [31]:
duplicate_html_block = ''

In [35]:
dmc_is_duplicate

[(Decimal('107069185'),
  'https://moto.av.by/bike/honda/vt/107069185',
  True,
  Decimal('109140984')),
 (Decimal('112861938'),
  'https://moto.av.by/bike/honda/vt/112861938',
  True,
  Decimal('109140984')),
 (Decimal('115369830'),
  'https://moto.av.by/bike/honda/vt/115369830',
  True,
  Decimal('118836270'))]

In [21]:
dmc_not_duplicate

[(Decimal('102269941'), 'https://moto.av.by/bike/honda/vt/102269941', False),
 (Decimal('102580148'), 'https://moto.av.by/bike/honda/vt/102580148', False),
 (Decimal('102596655'), 'https://moto.av.by/bike/honda/vt/102596655', False),
 (Decimal('102732121'), 'https://moto.av.by/bike/honda/vt/102732121', False),
 (Decimal('103056878'), 'https://moto.av.by/bike/honda/vt/103056878', False),
 (Decimal('103352837'), 'https://moto.av.by/bike/honda/vt/103352837', False),
 (Decimal('103352896'), 'https://moto.av.by/bike/honda/vt/103352896', False),
 (Decimal('103674120'), 'https://moto.av.by/bike/honda/vt/103674120', False),
 (Decimal('103674137'), 'https://moto.av.by/bike/honda/vt/103674137', False),
 (Decimal('104374174'), 'https://moto.av.by/bike/honda/vt/104374174', False),
 (Decimal('104607002'), 'https://moto.av.by/bike/honda/vt/104607002', False),
 (Decimal('109140984'), 'https://moto.av.by/bike/honda/vt/109140984', False),
 (Decimal('115962895'), 'https://moto.av.by/bike/honda/vt/115962